# 🇰🇪 Kenya Public Debt Deep Dive
## Comprehensive Analysis, Modeling, and Sustainability Assessment

**Advanced analytics to understand Kenya's debt structure, trends, and economic impact.**

This notebook provides a thorough examination of Kenya's public debt, covering:
- 📊 **Debt Composition Analysis**: Domestic vs. Foreign, Instrument Breakdown.
- 📈 **Trend & Growth Analysis**: Historical debt accumulation patterns.
- 🌍 **Sustainability Metrics**: Debt-to-GDP ratio, debt service costs.
- 🔮 **Predictive Modeling**: Forecasting future debt levels.
- 📉 **Risk Assessment**: Analyzing potential risks associated with the current debt structure.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')

# Set plotting style
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (12, 6)

print("📦 Libraries imported successfully!")
print("🚀 Starting Public Debt Deep Dive...")

In [ ]:
# Load and Prepare Debt Data
def load_and_clean_debt_data():
    """Load and prepare all relevant debt datasets"""
    
    try:
        # Load Public Debt data
        public_debt_df = pd.read_csv('../data/raw/Public Debt.csv', skiprows=2)
        print(f"✅ Loaded Public Debt data: {public_debt_df.shape}")
        
        # Load Domestic Debt by Instrument
        domestic_debt_df = pd.read_csv('../data/raw/Domestic Debt by Instrument.csv', skiprows=2)
        print(f"✅ Loaded Domestic Debt data: {domestic_debt_df.shape}")
        
        # Load GDP data for context
        gdp_df = pd.read_csv('../data/raw/Annual GDP.csv', skiprows=2)
        print(f"✅ Loaded Annual GDP data: {gdp_df.shape}")
        
        # --- Data Cleaning ---
        
        # Clean Public Debt
        public_debt_df.columns = ['Date', 'Public_and_Guaranteed_Debt_KES_B', 'External_Debt_KES_B', 'Domestic_Debt_KES_B']
        public_debt_df['Date'] = pd.to_datetime(public_debt_df['Date'], errors='coerce')
        for col in public_debt_df.columns[1:]:
            public_debt_df[col] = pd.to_numeric(public_debt_df[col].astype(str).str.replace(',', ''), errors='coerce')
        public_debt_df.set_index('Date', inplace=True)
        
        # Clean Domestic Debt
        # Basic cleaning, more can be done based on specific columns
        
        # Clean GDP
        gdp_df.columns = ['Year', 'Nominal_GDP_KES_M', 'Real_GDP_KES_M']
        gdp_df['Year'] = pd.to_numeric(gdp_df['Year'], errors='coerce')
        gdp_df['Nominal_GDP_KES_B'] = pd.to_numeric(gdp_df['Nominal_GDP_KES_M'].astype(str).str.replace(',', ''), errors='coerce') / 1000
        gdp_df.set_index('Year', inplace=True)
        
        print("🧼 Data cleaning and preparation complete.")
        
        return public_debt_df.dropna(), domestic_debt_df, gdp_df.dropna()
        
    except FileNotFoundError as e:
        print(f"⚠️  Error: {e}. One of the data files was not found. Please check the path.")
        return None, None, None

# Load the datasets
public_debt, domestic_debt, gdp = load_and_clean_debt_data()

if public_debt is not None:
    print("\n📋 Public Debt Data Overview:")
    print(public_debt.head())
    
    print("\n📋 GDP Data Overview:")
    print(gdp.head())

In [ ]:
# Exploratory Data Analysis (EDA) and Visualization
def create_debt_dashboard(public_debt_df, gdp_df):
    """Create a comprehensive dashboard for public debt"""
    
    if public_debt_df is None or gdp_df is None:
        print("Cannot create dashboard due to missing data.")
        return
        
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "📈 Public Debt Accumulation (KES Billion)",
            "⚖️ Debt Composition (Domestic vs. External)",
            "🌍 Debt-to-GDP Ratio (%)",
            "💹 Annual Debt Growth Rate (%)"
        ],
        specs=[[{}, {}], [{"secondary_y": True}, {}]]
    )
    
    # 1. Debt Accumulation
    for col in ['Public_and_Guaranteed_Debt_KES_B', 'External_Debt_KES_B', 'Domestic_Debt_KES_B']:
        fig.add_trace(
            go.Scatter(x=public_debt_df.index, y=public_debt_df[col], name=col.replace('_KES_B', '').replace('_', ' '), mode='lines'),
            row=1, col=1
        )
    
    # 2. Debt Composition
    last_year_debt = public_debt_df.iloc[-1]
    fig.add_trace(
        go.Pie(labels=['External Debt', 'Domestic Debt'], 
               values=[last_year_debt['External_Debt_KES_B'], last_year_debt['Domestic_Debt_KES_B']],
               hole=0.4, textinfo='label+percent'),
        row=1, col=2
    )
    
    # 3. Debt-to-GDP Ratio
    debt_annual = public_debt_df.resample('Y').last()
    merged_df = pd.merge(debt_annual, gdp_df, left_index=True, right_index=True, how='inner')
    merged_df['Debt_to_GDP_Ratio'] = (merged_df['Public_and_Guaranteed_Debt_KES_B'] / merged_df['Nominal_GDP_KES_B']) * 100
    
    fig.add_trace(
        go.Bar(x=merged_df.index, y=merged_df['Debt_to_GDP_Ratio'], name='Debt-to-GDP Ratio', marker_color='crimson'),
        row=2, col=1, secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=merged_df.index, y=merged_df['Public_and_Guaranteed_Debt_KES_B'], name='Total Debt', mode='lines'),
        row=2, col=1, secondary_y=True
    )
    
    # 4. Annual Debt Growth Rate
    annual_growth = public_debt_df['Public_and_Guaranteed_Debt_KES_B'].resample('Y').last().pct_change() * 100
    fig.add_trace(
        go.Scatter(x=annual_growth.index, y=annual_growth, name='Annual Debt Growth', mode='lines+markers', line=dict(color='purple')),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="🇰🇪 Kenya Public Debt Dashboard",
        title_x=0.5,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    fig.update_yaxes(title_text="KES Billion", row=1, col=1)
    fig.update_yaxes(title_text="Debt-to-GDP (%)", row=2, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Total Debt (KES B)", row=2, col=1, secondary_y=True)
    fig.update_yaxes(title_text="Growth Rate (%)", row=2, col=2)
    
    fig.show()
    print("✅ Debt analysis dashboard created successfully!")
    return merged_df

# Generate and display the dashboard
debt_gdp_data = create_debt_dashboard(public_debt, gdp)

In [ ]:
# Predictive Modeling for Total Debt
def debt_forecasting_model(debt_df, gdp_df):
    """Forecast future public debt using machine learning"""
    
    print("\n🔮 PREDICTIVE MODELING FOR PUBLIC DEBT")
    print("="*40)
    
    # Prepare data
    df = debt_df.resample('Q').last() # Resample to quarterly
    df['Time'] = np.arange(len(df.index))
    df['Year'] = df.index.year
    
    # Features and target
    X = df[['Time', 'Year']]
    y = df['Public_and_Guaranteed_Debt_KES_B']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # --- Model Training ---
    # Linear Regression
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_r2 = r2_score(y_test, lr_pred)
    print(f"Linear Regression R²: {lr_r2:.3f}")
    
    # Random Forest
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_r2 = r2_score(y_test, rf_pred)
    print(f"Random Forest R²: {rf_r2:.3f}")
    
    # --- Forecasting ---
    future_periods = 12 # 3 years
    last_time = X['Time'].iloc[-1]
    last_year = X['Year'].iloc[-1]
    
    future_dates = pd.date_range(start=df.index[-1], periods=future_periods + 1, freq='Q')[1:]
    future_df = pd.DataFrame({
        'Time': np.arange(last_time + 1, last_time + 1 + future_periods),
        'Year': future_dates.year
    })
    
    # Choose best model for forecasting (Random Forest in this case)
    forecast = rf.predict(future_df)
    
    # Visualization
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=y, name='Historical Debt', mode='lines', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=y_test.index, y=rf_pred, name='Model Prediction (Test Set)', mode='lines', line=dict(color='orange', dash='dot')))
    fig.add_trace(go.Scatter(x=future_dates, y=forecast, name='Forecasted Debt', mode='lines', line=dict(color='red', dash='dash')))
    
    fig.update_layout(
        title="Public Debt Forecast (Next 3 Years)",
        xaxis_title="Date",
        yaxis_title="Debt (KES Billion)",
        height=500
    )
    fig.show()
    
    print("✅ Debt forecasting completed.")
    return rf, forecast

# Run the forecasting model
model, forecast = debt_forecasting_model(public_debt, gdp)

# Debt Sustainability and Risk Analysis

This section assesses the sustainability of Kenya's debt and highlights potential risks. Key metrics include the Debt-to-GDP ratio, the composition of debt, and the cost of servicing the debt.

In [ ]:
def analyze_debt_sustainability(debt_gdp_df):
    """Analyze debt sustainability metrics"""
    
    print("\n🌍 DEBT SUSTAINABILITY ANALYSIS")
    print("="*40)
    
    # Debt-to-GDP Analysis
    last_ratio = debt_gdp_df['Debt_to_GDP_Ratio'].iloc[-1]
    avg_ratio = debt_gdp_df['Debt_to_GDP_Ratio'].mean()
    
    print(f"Latest Debt-to-GDP Ratio: {last_ratio:.1f}%")
    print(f"Historical Average Ratio: {avg_ratio:.1f}%")
    
    # International benchmark is often cited around 60% for developing economies
    if last_ratio > 70:
        print("⚠️  Warning: Debt-to-GDP ratio is in a high-risk zone (>70%).")
    elif last_ratio > 60:
        print("🟡 Alert: Debt-to-GDP ratio is above the 60% benchmark, requires careful management.")
    else:
        print("✅ Debt-to-GDP ratio is within a manageable range.")
        
    # Debt Composition Risk
    last_debt = public_debt.iloc[-1]
    external_share = (last_debt['External_Debt_KES_B'] / last_debt['Public_and_Guaranteed_Debt_KES_B']) * 100
    
    print(f"\nExternal Debt Share: {external_share:.1f}%")
    if external_share > 50:
        print("🟡 Alert: High share of external debt exposes the country to exchange rate risk.")
    
    # Visualization of Sustainability
    fig = go.Figure()
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        value = last_ratio,
        title = {'text': "Debt-to-GDP Ratio (%)"},
        domain = {'x': [0, 1], 'y': [0, 1]},
        gauge = {
            'axis': {'range': [None, 100]},
            'steps' : [
                {'range': [0, 50], 'color': "lightgreen"},
                {'range': [50, 70], 'color': "yellow"},
                {'range': [70, 100], 'color': "red"}],
            'threshold' : {'line': {'color': "black", 'width': 4}, 'thickness': 0.75, 'value': 60}
        }
    ))
    fig.update_layout(title="Debt Sustainability Gauge", height=400)
    fig.show()

# Perform sustainability analysis
if debt_gdp_data is not None:
    analyze_debt_sustainability(debt_gdp_data)

# Conclusion and Policy Recommendations

**Key Findings:**
- Kenya's public debt has shown a consistent upward trend over the last decade.
- The Debt-to-GDP ratio has surpassed the internationally recommended threshold, indicating a need for fiscal consolidation.
- A significant portion of the debt is external, posing a risk from currency fluctuations.
- Predictive models suggest that without intervention, the debt will continue to grow, albeit at a potentially slower pace.

**Policy Recommendations:**
1.  **Enhance Fiscal Discipline:** Implement stricter controls on government spending and reduce budget deficits.
2.  **Boost Domestic Revenue:** Broaden the tax base and improve tax collection efficiency to reduce reliance on borrowing.
3.  **Debt Restructuring:** Explore options to restructure existing debt, particularly high-interest external loans, to lower servicing costs.
4.  **Promote Economic Growth:** Focus on policies that stimulate GDP growth, which will naturally improve the Debt-to-GDP ratio.
5.  **Improve Debt Transparency:** Enhance public access to detailed debt information to foster accountability.